In [20]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/huacre/UCI HAR Dataset/activity_labels.txt
/kaggle/input/huacre/UCI HAR Dataset/README.txt
/kaggle/input/huacre/UCI HAR Dataset/features_info.txt
/kaggle/input/huacre/UCI HAR Dataset/features.txt
/kaggle/input/huacre/UCI HAR Dataset/test/subject_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/X_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_ac

In [21]:
import IPython.display as ipd
import warnings
#warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout , Flatten

from sklearn.metrics import accuracy_score , confusion_matrix , classification_report

In [22]:
def boom1():
    beep = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
    return ipd.Audio(beep, rate=10000, autoplay=True)

def boom2():
    audio_path="https://www.mfiles.co.uk/mp3-downloads/cherry-ripe.mp3"
    return ipd.Audio(audio_path, autoplay=True)

In [23]:
def s():
    print("="*50)

def ts():
    t_start = time.time()
    print("Time started...")
    s()
    s()
    return t_start

t_total = 0
def te(t1):
    t_end = time.time()
    t_time = t_end - t1
    s()
    s()
    global t_total
    t_total = t_total + t_time
    print("Time taken:",round(t_time, 4),"seconds.")
    s()
    print("Total Time:",round(t_total, 2),"seconds.")

In [24]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/huacre/UCI HAR Dataset/activity_labels.txt
/kaggle/input/huacre/UCI HAR Dataset/README.txt
/kaggle/input/huacre/UCI HAR Dataset/features_info.txt
/kaggle/input/huacre/UCI HAR Dataset/features.txt
/kaggle/input/huacre/UCI HAR Dataset/test/subject_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/X_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_acc_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_acc_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_acc_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_acc_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_y_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_x_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/body_gyro_z_test.txt
/kaggle/input/huacre/UCI HAR Dataset/test/Inertial Signals/total_ac

In [25]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

#DATADIR = '../input/huacre/UCI HAR Dataset'

In [26]:
def _read_csv(filename):
    return pd.read_csv(filename , header = None , delim_whitespace = True)

def load_signal(subset):
    signal_data = []
   
    for signal in SIGNALS:
        filename = f'../input/huacre/UCI HAR Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signal_data.append(
            _read_csv(filename).to_numpy()) 
    return np.transpose(signal_data, (1, 2, 0))


def load_y(subset):
    filename = f'../input/huacre/UCI HAR Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

def load_data():
    
    X_train, X_test = load_signal('train'), load_signal('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [27]:
Xtrain , Xtest , Ytrain , Ytest = load_data()

In [28]:
print(Xtrain.shape)
print(Ytrain.shape)
print(Xtest.shape)
print(Ytest.shape)

(7352, 128, 9)
(7352, 6)
(2947, 128, 9)
(2947, 6)


In [29]:
# x = np.ones((4, 3, 2))
# x
# np.transpose(x, (1, 2, 0)).shape

# 4, 3, 2             9, 7352, 128
# 3, 2, 4             7352, 128, 9


In [30]:
model = Sequential()
model.add(LSTM(32,input_shape = (128,9)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(30 , activation = "relu"))
model.add(Dense(6, activation = "softmax" ))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 30)                990       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 186       
Total params: 6,552
Trainable params: 6,552
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(optimizer = "adam" , loss= "categorical_crossentropy" , metrics = ["accuracy"])

In [35]:
model.fit(Xtrain , Ytrain , epochs = 10 , batch_size = 20 , validation_data = (Xtest , Ytest))

Epoch 1/10
368/368 [==============================] - 22s 61ms/step - loss: 0.6505 - accuracy: 0.7160 - val_loss: 0.8571 - val_accuracy: 0.6634
Epoch 2/10
368/368 [==============================] - 23s 61ms/step - loss: 0.6131 - accuracy: 0.7251 - val_loss: 0.8356 - val_accuracy: 0.6753
Epoch 3/10
368/368 [==============================] - 22s 61ms/step - loss: 0.5863 - accuracy: 0.7379 - val_loss: 0.8274 - val_accuracy: 0.6902
Epoch 4/10
368/368 [==============================] - 22s 61ms/step - loss: 0.5693 - accuracy: 0.7409 - val_loss: 0.8182 - val_accuracy: 0.6987
Epoch 5/10
368/368 [==============================] - 23s 61ms/step - loss: 0.6211 - accuracy: 0.7341 - val_loss: 0.7896 - val_accuracy: 0.7095
Epoch 6/10
368/368 [==============================] - 23s 62ms/step - loss: 0.5416 - accuracy: 0.7705 - val_loss: 0.7462 - val_accuracy: 0.7329
Epoch 7/10
368/368 [==============================] - 23s 62ms/step - loss: 0.6020 - accuracy: 0.7484 - val_loss: 0.6833 - val_accuracy:

In [36]:
loss , accuracy = model.evaluate(Xtest, Ytest)

93/93 [==============================] - 1s 15ms/step - loss: 0.6231 - accuracy: 0.8012


In [37]:
print("Loss on test :",loss)
print("Accuracy on test :",round(accuracy*100,2),"%")

Loss on test : 0.6230677366256714
Accuracy on test : 80.12 %


In [32]:
# model.fit(Xtrain , Ytrain , epochs = 20 , batch_size = 20 , validation_data = (Xtest , Ytest))

Epoch 1/20
368/368 [==============================] - 25s 63ms/step - loss: 1.3788 - accuracy: 0.4375 - val_loss: 0.9767 - val_accuracy: 0.5870
Epoch 2/20
368/368 [==============================] - 22s 61ms/step - loss: 0.8364 - accuracy: 0.6127 - val_loss: 0.8338 - val_accuracy: 0.5996
Epoch 3/20
368/368 [==============================] - 22s 61ms/step - loss: 0.6830 - accuracy: 0.6683 - val_loss: 0.6657 - val_accuracy: 0.7051
Epoch 4/20
368/368 [==============================] - 23s 61ms/step - loss: 0.5717 - accuracy: 0.7444 - val_loss: 0.5471 - val_accuracy: 0.7703
Epoch 5/20
368/368 [==============================] - 22s 61ms/step - loss: 0.4792 - accuracy: 0.8037 - val_loss: 0.4380 - val_accuracy: 0.8449
Epoch 6/20
368/368 [==============================] - 22s 61ms/step - loss: 0.3797 - accuracy: 0.8576 - val_loss: 0.8253 - val_accuracy: 0.7201
Epoch 7/20
368/368 [==============================] - 23s 61ms/step - loss: 0.4167 - accuracy: 0.8360 - val_loss: 0.4128 - val_accuracy:

In [33]:
#loss , accuracy = model.evaluate(Xtest, Ytest)

93/93 [==============================] - 1s 15ms/step - loss: 0.8733 - accuracy: 0.6637


In [34]:
#print("Loss on test :",loss)
#print("Accuracy on test :",round(accuracy*100,2),"%")

Loss on test : 0.8732960820198059
Accuracy on test : 66.37 %
